In [1]:
import mpramnist
from mpramnist.DeepPromoter.dataset import DeepPromoterDataset

from mpramnist.models import HumanLegNet
from mpramnist.models import initialize_weights
from mpramnist.trainers import LitModel_DeepPromoter

from mpramnist import transforms as t
from mpramnist import target_transforms as t_t

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

import pytorch_lightning as L

In [2]:
BATCH_SIZE = 128
NUM_WORKERS = 103
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
train_transform = t.Compose([
    t.Seq2Tensor(),
    t.ReverseComplement(0.5)
])
val_test_transform = t.Compose([
    t.Seq2Tensor(),
])
train_dataset = DeepPromoterDataset(split = "train",transform = train_transform, root= "../data/")

val_dataset = DeepPromoterDataset(split = "val",transform = val_test_transform, root= "../data/")

test_dataset = DeepPromoterDataset(split = "test", transform = val_test_transform, root= "../data/") 

In [5]:
print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))

9000
1000
1884


In [6]:
# encapsulate data into dataloader form
train_loader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers = NUM_WORKERS)

val_loader = data.DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers = NUM_WORKERS)

test_loader = data.DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers = NUM_WORKERS)

In [7]:
in_channels = len(train_dataset[0][0])
out_channels = 1

In [9]:
model = HumanLegNet(in_ch=in_channels,
                     output_dim = out_channels,
                     stem_ch=64,
                     stem_ks=11,
                     ef_ks=9,
                     ef_block_sizes=[80, 96, 112, 128],
                     pool_sizes=[2,2,2,2],
                     resize_factor=4)
model.apply(initialize_weights)

seq_model = LitModel_DeepPromoter(model = model,
                           loss = nn.MSELoss(),
                           weight_decay = 1e-4, lr = 1e-3, print_each = 5)

In [10]:
# Initialize a trainer
trainer = L.Trainer(
    accelerator="gpu",
    devices=[0],
    max_epochs=1,
    gradient_clip_val=1,
    precision='16-mixed', 
    enable_progress_bar = True,
    num_sanity_val_steps=0
)

# Train the model
trainer.fit(seq_model,
            train_dataloaders = train_loader,
            val_dataloaders = val_loader)
trainer.test(seq_model, dataloaders = test_loader)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
2025-04-12 16:12:26.095847: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-12 16:12:26.116866: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one 

Training: |                                                                                       | 0/? [00:00…

Validation: |                                                                                     | 0/? [00:00…

`Trainer.fit` stopped: `max_epochs=1` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Testing: |                                                                                        | 0/? [00:00…

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │        148712352.0        │
│       test_pearson        │    0.06543812900781631    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 148712352.0, 'test_pearson': 0.06543812900781631}]

In [7]:
import pytorch_lightning as L
from torch.nn import functional as F

def pearson_correlation(x, y):
    vx = x - torch.mean(x, dim=0)
    vy = y - torch.mean(y, dim=0)
    pearsons = torch.sum(vx * vy, dim=0) / (torch.sqrt(torch.sum(vx ** 2, dim=0)) * torch.sqrt(torch.sum(vy ** 2, dim=0)) + 1e-10)
    return torch.mean(pearsons)
    
class Seq1Model(L.LightningModule):
    
    def __init__(self, in_ch, out_ch, print_each = 1, weight_decay = 1e-2, lr=3e-4):
        super().__init__()
        
        self.model = HumanLegNet(in_ch=in_ch,
                                 output_dim = out_ch,
                                 stem_ch=64,
                                 stem_ks=11,
                                 ef_ks=9,
                                 ef_block_sizes=[80, 96, 112, 128],
                                 pool_sizes=[2,2,2,2],
                                 resize_factor=4)
        self.model.apply(initialize_weights)
    
        self.print_each = print_each
        self.weight_decay = weight_decay
    
        self.loss = nn.MSELoss() 
        self.lr = lr

        self.pred = torch.empty(0).to(device)
        self.target = torch.empty(0).to(device)
        self.train_pred = torch.empty(0).to(device)
        self.train_target = torch.empty(0).to(device)
        
    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_nb):
        X, y = batch
        y_hat = self.model(X)
        loss = self.loss(y_hat, y)
        
        self.log("train_loss", loss, prog_bar=True)

        self.train_pred = torch.cat([self.train_pred, y_hat], dim = 0)
        self.train_target = torch.cat([self.train_target, y], dim = 0)
        return loss
        
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = self.loss(y_hat, y)
        
        self.log('val_loss', loss, prog_bar=True)

        self.pred = torch.cat([self.pred, y_hat], dim = 0)
        self.target = torch.cat([self.target, y], dim = 0)
        
    def on_validation_epoch_end(self):
    
        corr = pearson_correlation(self.pred.clone().detach(), self.target.clone().detach())
        corr_train = pearson_correlation(self.train_pred.clone().detach(), self.train_target.clone().detach())

        self.log("val_pearson", corr, prog_bar=True)
        self.log("train_pearson", corr_train)
        
        if (self.current_epoch + 1) % self.print_each == 0:

            res_str = f"| Epoch: {self.current_epoch} "
            res_str += f"| Val Loss: {self.trainer.callback_metrics['val_loss']:.5f} "
            res_str += f'| Val Pearson: {corr:.5f} '
            res_str += f'| Train Loss: {self.trainer.callback_metrics['train_loss']:.5f} '
            res_str += f'| Train Pearson: {corr_train:.5f} |'
            border = '-'*len(res_str)
            print("\n".join(['', border, res_str, border, '']))
        
        self.pred = torch.empty(0).to(device)
        self.target = torch.empty(0).to(device)

        self.train_pred = torch.empty(0).to(device)
        self.train_target = torch.empty(0).to(device)
        
    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)

        self.pred = torch.cat([self.pred, y_hat], dim = 0)
        self.target = torch.cat([self.target, y], dim = 0)
        
    def on_test_epoch_end(self):
        corr = pearson_correlation(self.pred, self.target)
        self.log('test_pearson deff pearson_correlation', corr, on_step=False, on_epoch=True, prog_bar=True)
        self.pred = torch.empty(0).to(device)
        self.target = torch.empty(0).to(device)
        
    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        x, y = batch
        y_hat = self.model(x)          
        
        return {"y": y.cpu().detach().float(), "pred": y_hat.cpu().detach().float()}
    
    def train_dataloader(self):
        return data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers = NUM_WORKERS, pin_memory=True)

    def val_dataloader(self):
        return data.DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers = NUM_WORKERS, pin_memory=True)
    
    def test_dataloader(self):
        return data.DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers = NUM_WORKERS, pin_memory=True)
    
    def configure_optimizers(self):
        
        self.optimizer = torch.optim.AdamW(self.parameters(),
                                               lr=self.lr,
                                               weight_decay = self.weight_decay)
        
        lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(self.optimizer,
                                                        max_lr=self.lr,
                                                        three_phase=False, 
                                                        total_steps=self.trainer.estimated_stepping_batches, # type: ignore
                                                        pct_start=0.3,
                                                        cycle_momentum =False)
        lr_scheduler_config = {
                    "scheduler": lr_scheduler,
                    "interval": "step",
                    "frequency": 1,
                    "name": "cycle_lr"
            }
            
        return [self.optimizer], [lr_scheduler_config]